In [9]:
import win32com.client
import os

outlook = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
inbox = outlook.GetDefaultFolder(6) #Folder 6 is the Inbox by default
messaggi = inbox.Items
email = "hackathon-web3@sia-partners.com"
oggetto = "RE: BlockDotUnity - Business Case  Hackathon H-W3B Luxembourg"
#messaggi.Sort("[ReceivedTime]", True)

# Use Restrict to filter emails from a specific sender and subject
filter_criteria = f"[SenderEmailAddress] = '{email}'"
filtered_messages = messaggi.Restrict(filter_criteria)

# Sort only filtered messages
filtered_messages.Sort("[ReceivedTime]", True)


# I will now look for all the emails that were sent by a predeffined email addresse and that have a predefined Sobject
for msg in filtered_messages: 
    print("ecco")
    if msg.Class==43: # 43 refers to the email property of a message in the inbox. Not all the things that we can find there will be emails.
        if msg.SenderEmailType=='EX':
            sender = msg.Sender.GetExchangeUser().PrimarySmtpAddress
        else:
            sender = msg.SenderEmailAddress
        obj = msg.Subject
        if sender == email and oggetto.strip().lower() == obj.strip().lower():
            print(msg)
            msg.saveAs("f.msg")

            print(8)

ecco
RE: BlockDotUnity - Business Case  Hackathon H-W3B Luxembourg 
8


In [7]:
pip install weasyprint


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------------------------- ---- 1.8/2.1 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: tinycss2
    Found existing installation: tinycss2 1.2.1
    Uninstalling tinycss2-1.2.1:
      Successfully uninstalled tinycss2-1.2.1
Note: you may need to restart the kernel to use updated packages.


In [8]:

# with pdf 
import os
import re
import sys
import win32com.client
import pdfkit

# Percorso dove salvare i PDF
percorso_salvataggio = r'C:\Users\Balestra\Desktop\cacc'

if not os.path.exists(percorso_salvataggio):
    os.makedirs(percorso_salvataggio)

# Configurazione di pdfkit
# Se wkhtmltopdf è nel PATH, non è necessario specificare il percorso
# Se non lo è, specifica il percorso completo
path_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'  # Aggiorna questo percorso se necessario
config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

# Connessione a Outlook
outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

# Accedi alla cartella Posta in arrivo
inbox = namespace.GetDefaultFolder(6)  # 6 corrisponde a Posta in arrivo

# Funzione per trovare la cartella ricorsivamente
def find_folder(parent_folder, target_name):
    for folder in parent_folder.Folders:
        if folder.Name.strip().lower() == target_name.strip().lower():
            return folder
        else:
            subfolder = find_folder(folder, target_name)
            if subfolder is not None:
                return subfolder
    return None

nome_cartella = 'prova'
sottocartella = find_folder(inbox, nome_cartella)

if sottocartella is None:
    print(f"Cartella '{nome_cartella}' non trovata sotto Posta in arrivo.")
    sys.exit()

# Itera su ogni email nella sottocartella
for item in sottocartella.Items:
    if item.Class == 43:  # MailItem
        soggetto = item.Subject
        # Pulisci il soggetto per creare un nome file valido
        nome_file = re.sub(r'[\\/*?:"<>|]', "", soggetto) + '.pdf'
        percorso_file_pdf = os.path.join(percorso_salvataggio, nome_file)
        
        # Ottieni il contenuto HTML dell'email
        html_content = item.HTMLBody
        
        # Verifica se l'email ha contenuto HTML
        if html_content:
            try:
                # Converti l'HTML in PDF usando pdfkit
                pdfkit.from_string(html_content, percorso_file_pdf, configuration=config)
                print(f"Email '{soggetto}' salvata come PDF.")
            except Exception as e:
                print(f"Errore nella conversione dell'email '{soggetto}': {e}")
        else:
            print(f"L'email '{soggetto}' non contiene contenuto HTML.")



-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'gobject-2.0-0': error 0x7e.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'gobject-2.0-0'

In [10]:
import os
import re
import sys
import win32com.client

# Percorso dove salvare i file .msg
percorso_salvataggio = r'C:\Users\Balestra\Desktop\cacc'

if not os.path.exists(percorso_salvataggio):
    os.makedirs(percorso_salvataggio)

# Connessione a Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Accedi alla cartella Posta in arrivo
inbox = outlook.GetDefaultFolder(6)  # 6 corrisponde a Posta in arrivo

# Funzione per trovare la cartella ricorsivamente
def find_folder(parent_folder, target_name):
    for folder in parent_folder.Folders:
        if folder.Name.strip().lower() == target_name.strip().lower():
            return folder
        else:
            subfolder = find_folder(folder, target_name)
            if subfolder is not None:
                return subfolder
    return None

nome_cartella = 'prova'  # Nome della sottocartella
sottocartella = find_folder(inbox, nome_cartella)

if sottocartella is None:
    print(f"Cartella '{nome_cartella}' non trovata sotto Posta in arrivo.")
    sys.exit()

# Itera su ogni email nella sottocartella
for item in sottocartella.Items:
    if item.Class == 43:  # 43 corrisponde a MailItem
        soggetto = item.Subject
        # Pulisci il soggetto per creare un nome file valido
        nome_file = re.sub(r'[\\/*?:"<>|]', "", soggetto) + '.msg'
        percorso_file_msg = os.path.join(percorso_salvataggio, nome_file)
        
        # Controlla se il file esiste già
        if not os.path.exists(percorso_file_msg):
            try:
                # Salva l'email come file .msg
                item.SaveAs(percorso_file_msg, 3)  # 3 corrisponde al formato olMSG
                print(f"Email '{soggetto}' salvata come '{nome_file}'.")
            except Exception as e:
                print(f"Errore nel salvataggio dell'email '{soggetto}': {e}")
        else:
            print(f"Il file '{nome_file}' esiste già. Salto l'email.")


Email 'There are 2 jobs matching your criteria' salvata come 'There are 2 jobs matching your criteria.msg'.
Email 'StudCorner: 4 FREE TICKETS - Songes d'une Nuit - Theatre' salvata come 'StudCorner 4 FREE TICKETS - Songes d'une Nuit - Theatre.msg'.
Email 'MFE-S3-IG: Precisions : Important mandatory events : September 24th and 26th 2024' salvata come 'MFE-S3-IG Precisions  Important mandatory events  September 24th and 26th 2024.msg'.


In [12]:
import os
import re
import sys
import win32com.client
import tkinter as tk
from tkinter import messagebox

def salva_email():
    # Percorso dove salvare i file .msg
    percorso_salvataggio = r'C:\Users\Balestra\Desktop\cacc'

    if not os.path.exists(percorso_salvataggio):
        os.makedirs(percorso_salvataggio)

    # Connessione a Outlook
    try:
        outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    except Exception as e:
        messagebox.showerror("Errore", f"Impossibile connettersi a Outlook: {e}")
        return

    # Accedi alla cartella Posta in arrivo
    inbox = outlook.GetDefaultFolder(6)  # 6 corrisponde a Posta in arrivo

    # Funzione per trovare la cartella ricorsivamente
    def find_folder(parent_folder, target_name):
        for folder in parent_folder.Folders:
            if folder.Name.strip().lower() == target_name.strip().lower():
                return folder
            else:
                subfolder = find_folder(folder, target_name)
                if subfolder is not None:
                    return subfolder
        return None

    nome_cartella = 'prova'  # Nome della sottocartella
    sottocartella = find_folder(inbox, nome_cartella)

    if sottocartella is None:
        messagebox.showerror("Errore", f"Cartella '{nome_cartella}' non trovata sotto Posta in arrivo.")
        return

    # Conta il numero di email salvate
    email_salvate = 0

    # Itera su ogni email nella sottocartella
    for item in sottocartella.Items:
        if item.Class == 43:  # 43 corrisponde a MailItem
            soggetto = item.Subject
            # Pulisci il soggetto per creare un nome file valido
            nome_file = re.sub(r'[\\/*?:"<>|]', "", soggetto) + '.msg'
            percorso_file_msg = os.path.join(percorso_salvataggio, nome_file)

            # Controlla se il file esiste già
            if not os.path.exists(percorso_file_msg):
                try:
                    # Salva l'email come file .msg
                    item.SaveAs(percorso_file_msg, 3)  # 3 corrisponde al formato olMSG
                    print(f"Email '{soggetto}' salvata come '{nome_file}'.")
                    email_salvate += 1
                except Exception as e:
                    print(f"Errore nel salvataggio dell'email '{soggetto}': {e}")
            else:
                print(f"Il file '{nome_file}' esiste già. Salto l'email.")

    messagebox.showinfo("Completato", f"Processo completato. Email salvate: {email_salvate}")

# Creazione dell'interfaccia Tkinter
root = tk.Tk()
root.title("Salva Email Outlook")

# Dimensioni e posizione della finestra
root.geometry("300x150")
root.resizable(False, False)

# Etichetta informativa
label = tk.Label(root, text="Premi il pulsante per salvare le email.")
label.pack(pady=20)

# Pulsante per eseguire il codice
button = tk.Button(root, text="Salva Email", command=salva_email)
button.pack()

# Avvio dell'interfaccia
root.mainloop()
